In [ ]:
!pip install --upgrade openai
!pip install jupyter-black

In [1]:
%load_ext jupyter_black

<div style="display: flex; align-items: center;">
  <div style="flex: 1;">
    <img src="images/notebook.gif" alt="segment" width="500">
  </div>
  <div style="flex: 1;">
    <h2> Practical Prompt Engineering </h2>
    <br>
    <div>
       In this tutorial, you’ll learn how to:
       <ul>
        <li>Apply prompt engineering techniques to practical, real-world examples
        <li>Tap into the power of roles in messages to go beyond using singular role prompts
        <li>Use numbered steps, delimiters, few-shot prompting and other techniques to improve your results
        <li>Understand and use chain-of-thought prompting to add more context
      </ul>
    <div>
</div>

<div class="alert alert-block alert-warning">

⚠️ You can follow along by opening this notebook on google collab.

</div>

First let's import all the libraries we're going to use...

In [352]:
import os, json
import openai

In [ ]:
from google.colab import userdata

and retrieve our OpenAI API key...

In [64]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
# openai.api_key = userdata.get('OPENAI_API_KEY')

Chat completion API [documentation](https://platform.openai.com/docs/guides/gpt):

In [105]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
)

<div class="alert alert-block alert-info">

<b>Tip: </b>The 3 types of roles you can use are: <br>

<ul>
<li> <b>System role:</b> Allows you to specify the way the model answers questions. <br>
    Typically we can use it to determine what <b>role</b> the AI should play and how it should behave generally. <br> 

<li> <b>User role:</b> Equivalent to the queries made by the user

<li> <b>Assistant role:</b> The model’s responses
</ul>

</div>

Chat completions [response](https://platform.openai.com/docs/guides/gpt/chat-completions-response-format) format:

In [97]:
response = {
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "message": {
                "content": "The Los Angeles Dodgers won the World Series in 2020.",
                "role": "assistant",
            },
        }
    ],
    "created": 1677664795,
    "id": "chatcmpl-7QyqpwdfhqwajicIEznoc6Q47XAyW",
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "usage": {"completion_tokens": 17, "prompt_tokens": 57, "total_tokens": 74},
}

The assistant’s reply can be extracted with:

In [106]:
content = response["choices"][0]["message"]["content"]

The next function allows us to send prompts to the model and get a response back. <br>

We have specified the following model parameters:
* Model: **GPT-3.5-turbo**
* Temperature: **0**

The function takes as inputs <u>prompt</u> and a <u>list of previous messages</u>, and returns a new list of messsages that include the prompt and the response from the model:

In [263]:
def get_completion_from_messages(
    prompt, messages, model="gpt-3.5-turbo", temperature=0
):
    if not isinstance(messages, list):
        messages = [messages]

    messages.append(prompt)

    response = openai.ChatCompletion.create(
        model=model, messages=messages, temperature=temperature
    )

    message = response.choices[0]["message"]

    print(message["content"])

    messages.append(message)

    return messages

Let's test it!

In [261]:
messages = [
    {"role": "user", "content": "My name is Alex"},
    {
        "role": "assistant",
        "content": "Nice to meet you, Alex! How can I assist you today?",
    },
]

prompt = {"role": "user", "content": "What is my name?"}

response = get_completion_from_messages(prompt, messages)

Your name is Alex.


# Ways to Engineer your Prompts ⚙️

We will now look at a few different examples of how you can improve your promtps to become the utlimate prompt engineer!

Let's explore the following techniques:

<ul>
<li> <b>Zero-shot prompting</b> 
<li> <b>Role prompting</b> 
<li> <b>Detail & Specificity</b> 
<li> <b>Few-shot prompting</b> 
<li> <b>Using delimiters</b> 
<li> <b>Chain-of-thought prompting (CoT)</b> 
</ul>

## 1. Zero-shot prompting 💬

First, we have a list of reviews written below. Our task is to summarise them using ChatGPT. <br> Let's see what we can do.

In [37]:
reviews = [
    (
        "Date: December 15, 2021; Username: John123; Review: I am absolutely delighted \
    with this savings product! The interest rates are fantastic, and it has helped \
    me grow my savings significantly. Highly recommend!"
    ),
    (
        "Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite \
    disappointed with this savings product. The promised returns were not as \
    impressive as advertised, and the fees associated with it added up quickly. \
    Not worth it."
    ),
    (
        "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather \
    indifferent about this savings product. It's just like any other basic savings \
    account out there. Nothing special, but it does the job."
    ),
]

In [112]:
content = f"""Summarise these 3 reviews:{reviews}"""

print(content)

Summarise these 3 reviews:['Date: December 15, 2021; Username: John123; Review: I am absolutely delighted     with this savings product! The interest rates are fantastic, and it has helped     me grow my savings significantly. Highly recommend!', 'Date: November 28, 2021; Username: Sarah77; Review: I must say, I am quite     disappointed with this savings product. The promised returns were not as     impressive as advertised, and the fees associated with it added up quickly.     Not worth it.', "Date: January 5, 2022; Username: AlexSmith; Review: My opinion? I'm rather     indifferent about this savings product. It's just like any other basic savings     account out there. Nothing special, but it does the job."]


In [48]:
prompt = {
    "role": "user",
    "content": content,
}

messages = get_completion_from_messages(prompt, [])

Review 1: John123 is extremely satisfied with the savings product, praising the fantastic interest rates and significant growth of their savings. Highly recommends it.

Review 2: Sarah77 is disappointed with the savings product, as the promised returns were not as impressive as advertised and the associated fees accumulated quickly. Considers it not worth it.

Review 3: AlexSmith expresses indifference towards the savings product, considering it similar to any other basic savings account. It is deemed as nothing special but gets the job done.


Great! It provided a summary of each review.

What we just did is call **zero-shot prompting**, which is just a fancy way of saying that you’re asking a normal question or simply describing a task.

But it's safe to say this is by no means useful enough to be a product yet. 

Let's now dive into the other prompt engineering techniques to improve our results. 

## 2. Role Prompting 🤖

In the next example we are setting the **role** of the model via the system message. 

This is known as "Role Prompting", and is a general practice to help set the tone and context for the model's responses.

In [113]:
system_message = {
    "role": "system",
    "content": (
        """
        You are a Review Summariser.
        Your job is to process reviews from customers,
        and summarise them for analysis for the customer review team.
        """
    ),
}

messages = [system_message]

In [114]:
prompt = {
    "role": "user",
    "content": content,
}

messages = get_completion_from_messages(prompt, messages)

Review 1: John123 is extremely satisfied with the savings product. They highlight the fantastic interest rates and how it has significantly helped them grow their savings. They highly recommend it.

Review 2: Sarah77 is disappointed with the savings product. They mention that the promised returns were not as impressive as advertised and the associated fees added up quickly. They believe it is not worth it.

Review 3: AlexSmith is indifferent about the savings product. They state that it is just like any other basic savings account and nothing special. However, they mention that it does the job.

Overall, the reviews are mixed. While John123 highly recommends the product, Sarah77 is disappointed with it. AlexSmith is neutral and finds it to be an average savings account.


<div class="alert alert-block alert-success">

⚠️ By using a role prompt via the system message, the summary is more structured and much clearer to read for the user.
    
</div>

## 3. Detail and Specificity 🔍

It is important that when you prompt, you are specific and as clear as possible.

Note that clear ≠ short: the more detail you add, the better the model will understand how you want it to behave.

In the next example, we'll try to add a category for each review: positive, neutral or negative:

In [115]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. \
    Your job is to process reviews from customers, \
    and summarise them for analysis for the customer review team.
    Categorise each review as positive, neutral, or bad.
    """,
}

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

Review 1: Positive
Review 2: Bad
Review 3: Neutral


As you can see, the model has only output the categories, and we're now missing all the other information. 

Let's be a bit more specific:

In [116]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. Your job is to process reviews 
    from customers and summarise them for analysis for the customer review team.

    For each review output the name and date. 
    Also label each review as either 'Positive','Neutral', or 'Negative'.
    Then provide a summary of key points in a single sentence.
    """,
}

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

Review 1:
- Name: John123
- Date: December 15, 2021
- Sentiment: Positive
- Summary: John123 is delighted with the savings product, praising the fantastic interest rates and significant growth of savings.

Review 2:
- Name: Sarah77
- Date: November 28, 2021
- Sentiment: Negative
- Summary: Sarah77 is disappointed with the savings product, mentioning that the promised returns were not as impressive as advertised and the associated fees added up quickly.

Review 3:
- Name: AlexSmith
- Date: January 5, 2022
- Sentiment: Neutral
- Summary: AlexSmith is indifferent about the savings product, stating that it is similar to any other basic savings account and nothing special, but it gets the job done.


Great! We now have all the information displayed appropriately for each review. 

It's even structured the data for us in a list! 

Let's see if we can further increase out specificity by numbering the steps as well as specifying the format that we want the model to use for our output:

In [103]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. Your job is to process reviews 
    from customers and summarise them for analysis for the customer review team:

    1. For each review output the name and date. 
    2. Also label each review as either 'Positive','Neutral', or 'Negative'.
    3. Provide a summary of key points in a single sentence.

    Output: When outputting this information, use the following structure:
    - [Username]: name of customer who made review
    - [Date]: date of review, written as dd/mm/yyyy
    - [Sentiment]: review sentiment
    - [Key points]: key points of each review in one
    """,
}

In [117]:
content = f"""Summarise these 3 reviews:{reviews} and return your response as a JSON"""

prompt = {
    "role": "user",
    "content": content,
}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

{
  "reviews": [
    {
      "name": "John123",
      "date": "December 15, 2021",
      "sentiment": "Positive",
      "summary": "Delighted with the savings product, fantastic interest rates, highly recommend."
    },
    {
      "name": "Sarah77",
      "date": "November 28, 2021",
      "sentiment": "Negative",
      "summary": "Disappointed with the savings product, returns not as advertised, high fees."
    },
    {
      "name": "AlexSmith",
      "date": "January 5, 2022",
      "sentiment": "Neutral",
      "summary": "Indifferent about the savings product, similar to other basic savings accounts."
    }
  ]
}


As you can see, we get the output in JSON format. 

This is quite powerful: when we structure our output as we have done in the previous section, the model can easily convert this into a structured format as shown.

## 4. Few-shot prompting 📄

Few-shot prompting is a prompt engineering technique where you provide example tasks and their expected outputs in your prompt. 

So, instead of just describing the task you also provide examples:

In [144]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. Your job is to process reviews 
    from customers and summarise them for analysis for the customer review team.

    For each review output the name and date. 
    Also label each review as either 'Positive','Neutral', or 'Negative'.
    Then provide a summary of key points in a single sentence.
    
    Example 1:
    "Date: April 1, 2022\nUsername: JaneDoe\nReview: I'm really not happy with this savings product. \
    The interest rates are lower than promised and the customer service is lacking. I wouldn't recommend it.",

    Output 1:
    - username: "JaneDoe"
    - review_date: "01/04/2022"
    - sentiment: "negative"
    - key_points: "Lower interest rates than promised, lacking customer service"
    - summary: JaneDoe is unhappy with the savings product, stating that the interest \
    rates are lower than promised and the customer service is lacking.
    """,
}

In [145]:
content = f"""Summarise these 3 reviews:{reviews}"""

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

- username: "John123"
- review_date: "15/12/2021"
- sentiment: "positive"
- key_points: "Fantastic interest rates, significant savings growth"
- summary: John123 is delighted with the savings product, praising the fantastic interest rates and significant savings growth.

- username: "Sarah77"
- review_date: "28/11/2021"
- sentiment: "negative"
- key_points: "Disappointing returns, high fees"
- summary: Sarah77 is disappointed with the savings product, expressing dissatisfaction with the returns not meeting expectations and the accumulation of high fees.

- username: "AlexSmith"
- review_date: "05/01/2022"
- sentiment: "neutral"
- key_points: "Indifferent, basic savings account"
- summary: AlexSmith is indifferent about the savings product, describing it as a basic savings account without any standout features.


<div class="alert alert-block alert-success">

⚠️ By using a few-shot prompting, the model included the 'key_points' in its answer but also adopted our preferred way of writing the summary.
    
</div>

## 5. Using Delimiters 🔖

Delimiters can help to separate the content and examples from the task description. They can also make it possible to refer to specific parts of your prompt at a later point in the prompt. A delimiter can be any sequence of characters that usually wouldn’t appear together, for example:

* \>>>>>
* \====
* \####

The number of characters that you use doesn’t matter too much, as long as you make sure that the sequence is relatively unique, otherwise this might confuse the model. Additionally, you can add labels just before or just after the delimiters:

* START CONTENT>>>>> content <<<<<END CONTENT
* \==== START content END ====
* \#### START EXAMPLES examples #### END EXAMPLES

In [146]:
system_message = {
    "role": "system",
    "content": """
    You are a Review Summariser. Your job is to process reviews 
    from customers and summarise them for analysis for the customer review team.

    For each review output the name and date. 
    Also label each review as either 'Positive','Neutral', or 'Negative'.
    Then provide a summary of key points in a single sentence.
    
    #### START EXAMPLES ####

    ------ Example Inputs ------
    "Date: April 1, 2022\nUsername: JaneDoe\nReview: I'm really not happy with this savings product. \
    The interest rates are lower than promised and the customer service is lacking. I wouldn't recommend it.",

    ------ Example Outputs ------
    - username: "JaneDoe"
    - review_date: "01/04/2022"
    - sentiment: "negative"
    - key_points: "Lower interest rates than promised, lacking customer service"
    - summary: JaneDoe is unhappy with the savings product, stating that the interest \
    rates are lower than promised and the customer service is lacking.
    
    #### END EXAMPLES ####

    """,
}

In [147]:
content = f"""Summarise these 3 reviews:{reviews}"""

prompt = {"role": "user", "content": content}

messages = [system_message]
messages = get_completion_from_messages(prompt, messages)

- username: "John123"
  - review_date: "15/12/2021"
  - sentiment: "positive"
  - key_points: "Fantastic interest rates, helped grow savings significantly"
  - summary: John123 is delighted with the savings product, praising the fantastic interest rates and how it has significantly helped grow their savings.

- username: "Sarah77"
  - review_date: "28/11/2021"
  - sentiment: "negative"
  - key_points: "Promised returns not as impressive as advertised, fees added up quickly"
  - summary: Sarah77 is disappointed with the savings product, expressing that the promised returns were not as impressive as advertised and the fees associated with it added up quickly.

- username: "AlexSmith"
  - review_date: "05/01/2022"
  - sentiment: "neutral"
  - key_points: "Indifferent, just like any other basic savings account"
  - summary: AlexSmith is indifferent about the savings product, stating that it is just like any other basic savings account out there and nothing special, but it does the job.


# Chain-of-thought prompting (CoT) 💭

To apply CoT, you prompt the model to generate intermediate results that then become part of the prompt in a second request. The increased context makes it more likely that the model will arrive at a useful output.

The smallest form of CoT prompting is **zero-shot CoT**, where you literally ask the model to *think step by step*. <br> This approach yields impressive results for mathematical tasks that LLMs otherwise often solve incorrectly.

More commonly, chain-of-thought operations are technically split into two stages:

- *Reasoning extraction*, where the model generates the increased context
- *Answer extraction*, where the model uses the increased context to generate the answer

# Zero-shot CoT

In [353]:
content = f"""
I went to the market and bought 10 apples.
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step.
"""

prompt = {"role": "user", "content": content}

messages = []
messages = get_completion_from_messages(prompt, messages)

Step 1: Bought 10 apples.
Step 2: Gave 2 apples to the neighbor. Remaining: 10 - 2 = 8 apples.
Step 3: Gave 2 apples to the repairman. Remaining: 8 - 2 = 6 apples.
Step 4: Bought 5 more apples. Total: 6 + 5 = 11 apples.
Step 5: Ate 1 apple. Remaining: 11 - 1 = 10 apples.

Therefore, you remained with 10 apples.


# Summary 🔥

We have looked at all the main different techniques that you can use to engineer your prompts! 

<ul>
<li> <b>Zero-shot prompting:</b> Asking the language model a normal question without any additional context
<li> <b>Role prompting:</b> Specifying how the model will answer the questions
<li> <b>Detail & Specificity:</b> Breaking down a complex prompt into a series of small, specific steps
<li> <b>Few-shot prompting:</b> Conditioning the model on a few examples to boost its performance
<li> <b>Using delimiters:</b> Adding special tokens or phrases to provide structure and instructions to the model
<li> <b>Chain-of-thought prompting:</b> Prompt the model to generate intermediate results that then become part of the prompt in a second request

</ul>

# Bonus: Chain-of-thought prompting in functions 💥

Chain of thought process is a powerful technique that has enabled LLMs to interact with functions and itegrate the provided context into their answers.

Let's try to demonstrate this by answering the following question:

*What is the weather in Edinburgh?*

First we need to modify our API call to enable function calling:

In [327]:
def get_completion_from_messages_func(
    prompt, messages, functions, model="gpt-3.5-turbo-0613", temperature=0
):
    if not isinstance(messages, list):
        messages = [messages]

    messages.append(prompt)

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=temperature,
    )

    message = response.choices[0]["message"]

    print(message)

    messages.append(message)

    return messages

Now let's define our python API that returns the weather using a location as input:

In [350]:
def get_current_weather(location):
    if location == "Edinburgh":
        return {"temperature": 9, "unit": "celsius", "description": "Sunny"}
    else:
        return None

Remember! LLMs can only understand *language* so the only way for our model to have knowledge of this function is if we describe it:

In [342]:
function = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location/city eg. London",
                },
            },
            "required": ["location"],
        },
    }
]

Let's now send our question to the model:

In [343]:
content = f"""What is the weather in Edinburgh?"""

prompt = {"role": "user", "content": content}

messages = []

messages = get_completion_from_messages_func(prompt, messages, function)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Edinburgh\"\n}"
  }
}


First we need to extract the function name and function arguments from the response:

In [344]:
function_name = messages[-1]["function_call"]["name"]
function_args = json.loads(messages[-1]["function_call"]["arguments"])

In [354]:
print("Function Name:", function_name)
print("Function Arguments:", function_args)

Function Name: get_current_weather
Function Arguments: {'location': 'Edinburgh'}


Let's now call our actual python function to get the response!

In [351]:
function_response = eval(function_name)(**function_args)

print(function_response)

{'temperature': 9, 'unit': 'celsius', 'description': 'Sunny'}


**Note:**

This: ```eval(function_name)(**function_args)```

is equivalent to this: ```get_current_weather(location='Edinburgh')```

<div class="alert alert-block alert-info">

<b>Tip: </b>There is actually only more role that can be used with ONLY gpt-3.5-turbo-0613 and gpt-4-0613. <br>

<ul>
<li> <b>Function role:</b> Allows you to specify the response of a function that the model can use to answer the original question.
</ul>

</div>

In [347]:
new_message = {
    "role": "function",
    "name": function_name,
    "content": str(function_response),
}

Let's finally get the response from our model:

In [348]:
messages = get_completion_from_messages_func(new_message, messages, function)

{
  "role": "assistant",
  "content": "The current weather in Edinburgh is sunny with a temperature of 9 degrees Celsius."
}
